# Recommender Model Retraining Template
# This notebook can be run every time you want to retrain your model
# It automatically handles versioning, backup, testing, and deployment

## Workflow:
1. Backup current production model
2. Train new model version
3. Deploy to TEST endpoint
4. Validate test endpoint
5. Promote to production OR rollback

---

In [ ]:
# Cell 0: Setup and imports
!pip install "oracle-ads[recommender]" oracledb --quiet

import ads
import pandas as pd
import joblib
import json
from pathlib import Path
from ads.model import GenericModel
from ads.common.model_metadata import UseCaseType

# Import our deployment manager
import sys
sys.path.append('/home/datascience')
from recommender_deployment_manager import RecommenderDeploymentManager, print_backups

# Set authentication for OCI Data Science notebooks
ads.set_auth(auth='resource_principal')

print("✅ Setup complete")

In [ ]:
# Cell 1: Initialize Deployment Manager
manager = RecommenderDeploymentManager(
    project_name="Product Recommender",
    backup_root="/home/datascience/backups",
    artifact_dir="/home/datascience/recommender_model_artifact",
    results_dir="/home/datascience/results"
)

# Show current deployment status
print(manager.get_deployment_summary())

# Get next version number
next_version = manager.get_next_version()
print(f"🎯 Next version to be trained: v{next_version}")

In [ ]:
# Cell 2: Backup current production artifacts (if any)
current_version = manager.state['current_version']

if current_version > 0:
    backup_dir = manager.backup_current_artifacts(version=current_version)
    print(f"✅ Backed up v{current_version} to: {backup_dir}")
else:
    print("ℹ️  No previous version to backup (first deployment)")

In [ ]:
# Cell 3: Connect to database and extract data
# Replace with your connection details
connection_parameters = {
    "user_name": "ADMIN",
    "password": "your_password",  # TODO: Use secrets or config file
    "service_name": "your_service_medium",
    "wallet_location": "/path/to/wallet"
}

# Query interactions
interactions_query = """
    SELECT DISTINCT 
        USER_ID as user_id,
        PRODUCT_ID as product_id,
        RATING as rating,
        ORDER_CREATED_AT as timestamp
    FROM ADMIN.ORDERS_PROFILE_V
    WHERE RATING IS NOT NULL
    ORDER BY ORDER_CREATED_AT DESC
"""

# Query user demographics
users_query = """
    SELECT DISTINCT
        USER_ID as user_id,
        GENDER as gender,
        BIRTH_YEAR as birth_year,
        COUNTRY as country,
        DEVICE as device
    FROM ADMIN.ORDERS_PROFILE_V
    WHERE USER_ID IS NOT NULL
"""

print("📊 Extracting data from database...")

interactions_df = pd.read_sql(
    interactions_query,
    con=connection_parameters
)

users_df = pd.read_sql(
    users_query,
    con=connection_parameters
)

# Save to CSV for operator
interactions_df.to_csv("/home/datascience/interactions.csv", index=False)
users_df.to_csv("/home/datascience/users.csv", index=False)

num_users = len(users_df)
num_products = interactions_df['product_id'].nunique()
num_interactions = len(interactions_df)

print(f"✅ Data extracted:")
print(f"   Users: {num_users:,}")
print(f"   Products: {num_products:,}")
print(f"   Interactions: {num_interactions:,}")

In [ ]:
# Cell 4: Configure ADS Recommender Operator
recommender_config = f"""
kind: operator
version: v1
type: recommender
spec:
  interaction_data: /home/datascience/interactions.csv
  user_data: /home/datascience/users.csv
  output_dir:
    name: /home/datascience/results
  top_k: 10
  user_id: user_id
  item_id: product_id
  rating: rating
"""

with open('/home/datascience/recommender_config.yaml', 'w') as f:
    f.write(recommender_config)

print("✅ Recommender configuration saved")

In [ ]:
# Cell 5: Clean previous run results
import shutil

results_dir = Path("/home/datascience/results")
if results_dir.exists():
    shutil.rmtree(results_dir)
    print("🗑️  Cleaned previous results")

artifact_dir = Path("/home/datascience/recommender_model_artifact")
if artifact_dir.exists():
    shutil.rmtree(artifact_dir)
    print("🗑️  Cleaned previous artifacts")

print("✅ Ready for new training")

In [ ]:
# Cell 6: Run ADS Recommender Operator
print("🤖 Training recommender model...")
print("   This may take a few minutes...\n")

!ads operator run -f /home/datascience/recommender_config.yaml --backend local

print("\n✅ Training complete!")

In [ ]:
# Cell 7: Load recommendations and create model artifact
print("📦 Creating model artifact...")

# Load pre-computed recommendations
recommendations_df = pd.read_csv("/home/datascience/results/recommendations.csv")

print(f"   Loaded {len(recommendations_df)} recommendation rows")
print(f"   For {recommendations_df['user_id'].nunique()} users")

# Create dictionary for fast lookup
recommender_dict = {}
for user_id, group in recommendations_df.groupby('user_id'):
    recommender_dict[str(user_id)] = group[['product_id', 'score']].to_dict('records')

# Save model artifact
artifact_dir = Path("/home/datascience/recommender_model_artifact")
artifact_dir.mkdir(exist_ok=True)

joblib.dump(recommender_dict, artifact_dir / "recommender_model.pkl")

print(f"✅ Model artifact created with {len(recommender_dict)} users")

In [ ]:
# Cell 8: Prepare model for deployment
from ads.common.model_metadata import UseCaseType

print("🔧 Preparing model for deployment...")

# Create GenericModel
model = GenericModel(artifact_dir=str(artifact_dir))

# Custom score.py with proper error handling
score_py_content = '''
import json
import joblib

def load_model(model_file_name="recommender_model.pkl"):
    """Load the recommender dictionary."""
    return joblib.load(model_file_name)

def predict(data, model=load_model()):
    """Generate recommendations for a user."""
    user_id = str(data.get("user_id", ""))
    top_k = data.get("top_k", 10)
    
    if user_id not in model:
        return {
            "user_id": user_id,
            "recommendations": [],
            "message": "User not found in training data. No recommendations available."
        }
    
    recommendations = model[user_id][:top_k]
    
    return {
        "user_id": user_id,
        "recommendations": recommendations,
        "count": len(recommendations)
    }
'''

# Prepare with custom score.py
model.prepare(
    inference_conda_env="oci://service-conda-packs@ociodscdev/service_pack/cpu/Data_Exploration_and_Manipulation_for_CPU_Python_3.9/3.0/dataexpl_p39_cpu_v3",
    model_file_name="recommender_model.pkl",
    score_py_uri=None,
    use_case_type=UseCaseType.RECOMMENDER,
    force_overwrite=True,
    ignore_conda_error=True
)

# Write custom score.py
with open(artifact_dir / "score.py", 'w') as f:
    f.write(score_py_content)

print("✅ Model prepared for deployment")

In [ ]:
# Cell 9: Save model to Model Catalog (TEST version)
model_id = manager.save_new_model(
    model=model,
    version=next_version,
    num_users=num_users,
    num_products=num_products,
    is_test=True
)

print(f"✅ Model v{next_version} saved to catalog: {model_id}")

In [ ]:
# Cell 10: Deploy to TEST endpoint
test_deployment = manager.deploy_model(
    model=model,
    model_id=model_id,
    version=next_version,
    is_test=True,
    instance_shape="VM.Standard.E4.Flex",
    ocpus=1,
    memory_gb=16
)

print(manager.get_deployment_summary())

In [ ]:
# Cell 11: Test the new model
print("🧪 Testing new model...\n")

# Test with a few sample users
test_users = recommendations_df['user_id'].unique()[:3]

for user_id in test_users:
    result = test_deployment.predict({"user_id": str(user_id), "top_k": 5})
    print(f"User {user_id}:")
    print(json.dumps(result, indent=2))
    print()

# Test with a user not in training data
print("Testing with unknown user:")
result = test_deployment.predict({"user_id": "unknown_user_12345", "top_k": 5})
print(json.dumps(result, indent=2))

print("\n✅ Testing complete")

---
# ⚠️ DECISION POINT: Promote or Rollback

## Choose ONE of the following options:

### Option A: Promote to Production (run Cell 12)
If testing is successful, promote the new model to production

### Option B: Rollback (run Cell 13)
If there are issues, rollback to previous version

---

In [ ]:
# Cell 12: OPTION A - Promote to Production
# Run this cell ONLY if you want to promote the test model to production

print("🚀 Promoting to production...\n")

manager.promote_to_production()

# Clean up test deployment to save costs
manager.cleanup_test_deployment()

print(manager.get_deployment_summary())
print(f"\n✅ v{next_version} is now in PRODUCTION!")

In [ ]:
# Cell 13: OPTION B - Rollback
# Run this cell ONLY if you want to rollback and discard the test model

print("⏪ Rolling back...\n")

# Clean up test deployment
manager.cleanup_test_deployment()

# Rollback local artifacts to previous version
if current_version > 0:
    manager.rollback_artifacts(version=current_version)
    print(f"✅ Rolled back to v{current_version}")
else:
    print("ℹ️  No previous version to rollback to")

print(manager.get_deployment_summary())

In [ ]:
# Cell 14: View all backups (optional)
backups = manager.list_backups()
print_backups(backups)

In [ ]:
# Cell 15: Manual rollback to specific version (optional)
# Uncomment and run if you need to rollback to a specific older version

# target_version = 2  # Change this to the version you want
# manager.rollback_artifacts(version=target_version)
# print(f"✅ Rolled back to v{target_version}")